# Real-Time Quality Control of In-situ Temperature and Salinity

In this notebook, we run quality assurance tests on real-time oceanographic data using QARTOD packages developed by Integrated Ocean Observing System(IOOS). 

Link: https://ioos.noaa.gov/project/qartod/ 

## Description of QARTOD



## QC Flags:

| Flag | Description |
| :- |-------------: |
| Pass=1| Data have passed critical real-time quality control tets and are deemed for adequate for use as preliminary data|
| Not Evaluated=2| Data have not been QC-tested, or the information on quality is not available. |
| Suspect or Of High Interest=3| Data are considered to be either suspect or of high interest to data providers and users.|
| Fail=4| Data are considered to have failed one or more critical real-time QC checks.  |
| Missing Data=9| Data are missing; used as a placeholder. |

## Description of Tests used:

| Test | Description |
| :- |-------------: |
| Gross range test | Data point exceeds sensor or operator selected min/max. |
| Climatology test | Test that data point falls within seasonal expectations. |
| Spike Test | Data point n-1 exceeds a selected threshold relative to adjacent data points. |
| Rate of change test | Excessive rise/fall test. |
| Flat line test | Invariant value. |
| Attenuated Signal Test | Inadequate variation of the time series |

## Steps:

1. Load libraries
2. Select dataset to be used
3. Select column to run test against
4. Select start-end date
5. Load data
6. Select station
7. Set test parameters
8. Run Tests and View plots
9. Save configuration for future use

# Load libraries

In [ ]:
from pathlib import Path
basedir = Path().absolute()
libdir = basedir

import os
import datetime
import json
import numpy as np
import pandas as pd

#conda install -c conda-forge ioos_qc
from ioos_qc import qartod
from ioos_qc.config import QcConfig

from bokeh.layouts import gridplot
import bokeh
from bokeh.plotting import figure, output_file
from bokeh.io import output_notebook, show
output_notebook(hide_banner=True)


import ipywidgets as widgets
from ipywidgets import VBox, interactive
from IPython.display import display

def fetchDataFromERDDAP(dataset_id, variable_name, start_date, end_date):
    # Set ERDDAP server details
    s = 'http://10.11.1.82/erddap'
    p = 'tabledap'
    r = 'csv'
    
    metadata = ['station_id', 'time']

    param = [variable_name]

    # Generate parameter component of URL
    plist = ''
    for item in metadata + param:
        plist = plist+item+'%2C'
    plist = plist[0:-3]
    
    # Create dataframe for population
    df = pd.DataFrame()

    # Create ERDDAP url and load data for selected dates  
    url = s+"/"+p+"/"+dataset_id+"."+r+"?"+plist+"&time%3E="+start_date+"T00:00:00Z&time%3C"+end_date+"T23:59:59Z"
    df= pd.read_csv(url,header=[0],skiprows=[1],parse_dates=True,infer_datetime_format=True)

    #Replace all NaN values with -5
    #df = df.fillna(-5)
    
    return df

# Method to plot QC results using Bokeh
def plot_results(data, results, var_name, title):
    
    time = data.index
    obs = data[var_name]
    qc_test = results

    qc_pass = np.ma.masked_where(qc_test != 1, obs)
    qc_suspect = np.ma.masked_where(qc_test != 3, obs)
    qc_fail = np.ma.masked_where(qc_test != 4, obs)
    qc_notrun = np.ma.masked_where(qc_test != 2, obs)

    p = figure(x_axis_type="datetime", title=title, plot_width=900, plot_height=500)
    p.grid.grid_line_alpha=0.3
    p.xaxis.axis_label = 'Time'
    p.yaxis.axis_label = 'Observation Value'

    p.line(time, obs,  legend_label='obs', color='#A6CEE3')
    p.circle(time, qc_notrun, size=2, legend_label='qc not run', color='gray', alpha=0.2)
    p.circle(time, qc_pass, size=4, legend_label='qc pass', color='green', alpha=0.5)
    p.circle(time, qc_suspect, size=4, legend_label='qc suspect', color='orange', alpha=0.7)
    p.circle(time, qc_fail, size=6, legend_label='qc fail', color='red', alpha=1.0)
    
    show(p)
    #return p

def range_slider(r,desc, minR, maxR):
    slider= widgets.FloatRangeSlider(
        value=r,
        min=minR,
        max=maxR,
        step=1,
        description=desc,
        disabled=False,
        continuous_update=False,
        orientation='horizontal',
        readout=True,
        readout_format='.0f',
        )
    return slider

def freq_slider(val,desc,st,minR,maxR,f):    
    freq_slider = widgets.FloatSlider(
        value=val,
        min=minR,
        max=maxR,
        step=st,
        description=desc,
        readout_format=f,
    )
    return freq_slider

def check_box(desc):
    check_box = widgets.Checkbox(
        value=False,
        description=desc,
        disabled=False,
        indent=False
    )
    return check_box

def gross_range(f,s):
    qc_config['qartod']['gross_range_test']['fail_span'] = f
    qc_config['qartod']['gross_range_test']['suspect_span'] = s
    
    run(qc_config,data_full,test = 'gross_range_test')
    
def climatology(a,b,c,d):
    qc_config['qartod']['climatology_test']['config'][0]['vspan'] = a
    qc_config['qartod']['climatology_test']['config'][1]['vspan'] = b
    qc_config['qartod']['climatology_test']['config'][2]['vspan'] = c
    qc_config['qartod']['climatology_test']['config'][3]['vspan'] = d
    
    run(qc_config,data_full,test = 'climatology_test')
    
def flat_line(t,s,f):
    
    print("Suspect Hours:",s)
    
    qc_config['qartod']['flat_line_test']['tolerance'] = t
    qc_config['qartod']['flat_line_test']['suspect_threshold'] = int(s*3600)
    qc_config['qartod']['flat_line_test']['fail_threshold'] = int(f*3600)
    
    run(qc_config,data_full,test = 'flat_line_test')
    
def rate_of_change(t,c):
    
    qc_config['qartod']['rate_of_change_test']['threshold'] = t
    df = data_full.copy()
    
    if c == True:
        #df.loc[df['gross_range_test']==4, variable_name] = np.nan 
        df = df[df['gross_range_test']!=4]
        data_full['rate_of_change_test'] = 0
    
    run(qc_config,df,test = 'rate_of_change_test')
    
def spike(s,f,c):
    qc_config['qartod']['spike_test']['suspect_threshold'] = s
    qc_config['qartod']['spike_test']['fail_threshold'] = f
    
    df = data_full.copy()
    
    if c == True:
        #df.loc[df['gross_range_test']==4, variable_name] = np.nan 
        df = df[df['gross_range_test']!=4]
        data_full['spike_test'] = 0
    
    run(qc_config,df,test = 'spike_test')
    
def att_signal(s,f,tp,mp):
    qc_config['qartod']['attenuated_signal_test']['suspect_threshold'] = s
    qc_config['qartod']['attenuated_signal_test']['fail_threshold'] = f
    qc_config['qartod']['attenuated_signal_test']['test_period'] = int(tp*3600)
    qc_config['qartod']['attenuated_signal_test']['min_period'] = int(mp*3600)
    
    run(qc_config,data_full,test = 'attenuated_signal_test')
    
def aggregate():
    run(qc_config,data_full,test = 'aggregate')
      
def run(qc_config, data, test):
    qc = QcConfig(qc_config)
    
    qc_results =  qc.run(
                inp=data[variable_name],
                tinp=data.index.values,
                zinp=np.ones(len(data[variable_name]))
            )

    results = qc_results['qartod'][test]
    
    if (test=='aggregate'):
        for t in qc_config['qartod'].keys():
            data[t] = qc_results['qartod'][t].data
    else:
        data[test] = qc_results['qartod'][test].data
        data['aggregate'] = qc_results['qartod']['aggregate'].data
        
    data_full.update(data)
    plot_results(data, results, variable_name, test.title())

# QC configuration
with open('Config/config_path.json') as f:
    config_dict = json.load(f)
    
print("Completed!")

# Select column to run tests against

In [ ]:
dataset = "Compass Mace Head Observation Buoy"

dataset_id = config_dict[dataset]["dataset_id"]
metadata = config_dict[dataset]["metadata"]

columns = config_dict[dataset]["columns"].keys()

variable_type = widgets.Dropdown(
    options=columns,
    disabled=False,
)

print("Select column to use:")
display(variable_type)

# Select start - end date

In [ ]:
start_date_type = widgets.DatePicker(
    description='Start Date',
    disabled=False
)
end_date_type = widgets.DatePicker(
    description='End Date',
    disabled=False
)
display(start_date_type)
display(end_date_type)

# Load data

In [ ]:
variable_name = str(variable_type.value)
start_date = str(start_date_type.value)
end_date = str(end_date_type.value)

data = fetchDataFromERDDAP(dataset_id, variable_name, start_date, end_date)

# Make a copy of the unaltered data download and save as csv
filename = dataset_id+'_'+'full'+'.csv'
filepath = basedir.joinpath('Datasets/'+filename)
data.to_csv(filepath, index=False)
print("Full resolution data downloaded. Available at '"+str(filepath)+"'.")
data_full = pd.read_csv(filepath, index_col='time', parse_dates=True)

# Load Config
configFilename = config_dict[dataset]["columns"][variable_name]
config_file = basedir.joinpath('Config/'+configFilename)
with open(config_file) as f:
    qc_config = json.load(f)
    

# Populate dataframe with qc columns
qc_columns = list(qc_config['qartod'].keys())
data_full[qc_columns] = 0

# Run Tests

It is recommened the tests are run sequentially!

In [ ]:
#Run Test and update paramters 

with open(config_file) as f:
    qc_config = json.load(f)

## Aggregate

tab1 = interactive(aggregate,{'manual': True, 'manual_name':'Run'})

##Gross Range Test
fail_span = qc_config['qartod']['gross_range_test']['fail_span']
suspect_span = qc_config['qartod']['gross_range_test']['suspect_span']
fail = range_slider(fail_span,'Fail:',-10, 30)
suspect = range_slider(suspect_span,'Suspect:',0, 25)
grList = [fail,suspect]

gr = interactive(gross_range,{'manual': True, 'manual_name':'Run'}, f=grList[0],s=grList[1]);
tab2 = VBox(children = gr.children)
#tab1 = VBox(children=grList)

##Climatology
climatology_config  = qc_config['qartod']['climatology_test']['config']
cList = []
for i in range(len(climatology_config)):
    cList.append(range_slider(climatology_config[i]['vspan'],'Period:'+str(climatology_config[i]['tspan']), 5, 20))
    
cl = interactive(climatology,{'manual': True, 'manual_name':'Run'}, a=cList[0],b=cList[1],c=cList[2],d=cList[3]);
tab3 = VBox(children = cl.children)
    
##Flat Line Test
tolerance = qc_config['qartod']['flat_line_test']['tolerance']
suspect_threshold = qc_config['qartod']['flat_line_test']['suspect_threshold']
fail_threshold = qc_config['qartod']['flat_line_test']['fail_threshold']

tol = freq_slider(tolerance,'Tolerance',0.0001,0,0.001,'.4f')
#suspect = freq_slider(suspect_threshold,'Suspect',1000,1000,10000,'.1f')
#fail = freq_slider(fail_threshold,'Fail',1000,10000,40000,'.1f')

# Hours
suspect = freq_slider(suspect_threshold/3600,'Suspect',0.5,0,5,'.1f')
fail = freq_slider(fail_threshold/3600,'Fail',0.5,0,10,'.1f')


flList = [tol,suspect,fail]

fl = interactive(flat_line,{'manual': True, 'manual_name':'Run'}, t=flList[0],s=flList[1],f=flList[2]);
tab4 = VBox(children = fl.children)

##Spike
suspect_threshold = qc_config['qartod']['spike_test']['suspect_threshold']
fail_threshold = qc_config['qartod']['spike_test']['fail_threshold']

suspect = freq_slider(suspect_threshold,'Suspect',0.2,0,10,'.1f')
fail = freq_slider(fail_threshold,'Fail',0.2,0,10,'.1f')
gr_check = check_box('Exclude records that failed Gross Range test')

sList = [suspect,fail, gr_check]

s = interactive(spike,{'manual': True, 'manual_name':'Run'}, s=sList[0],f=sList[1],c=sList[2]);
tab5 = VBox(children = s.children)

#Rate of Change
threshold = qc_config['qartod']['rate_of_change_test']['threshold']
t = freq_slider(threshold,'Threshold',0.001,0,0.01,'.3f')
gr_check = check_box('Exclude records that failed Gross Range test')
rctList = [t, gr_check]

rct = interactive(rate_of_change,{'manual': True, 'manual_name':'Run'}, t=rctList[0], c=rctList[1]);
tab6 = VBox(children = rct.children)

##Attenuated Signal
suspect_threshold = qc_config['qartod']['attenuated_signal_test']['suspect_threshold']
fail_threshold = qc_config['qartod']['attenuated_signal_test']['fail_threshold']
test_period = qc_config['qartod']['attenuated_signal_test']['test_period']
min_period = qc_config['qartod']['attenuated_signal_test']['min_period']

suspect = freq_slider(suspect_threshold,'Suspect',0.001,0,0.01,'.3f')
fail = freq_slider(fail_threshold,'Fail',0.001,0,0.01,'.3f')
# Hours
test_period = freq_slider(test_period/3600,'Test Period',0.5,0,30,'.1f')
min_period = freq_slider(min_period/3600,'Min Period',0.5,0,15,'.1f')

asList = [suspect,fail,test_period,min_period]

ats = interactive(att_signal,{'manual': True, 'manual_name':'Run'}, s=asList[0], f=asList[1], tp=asList[2], mp=asList[3]);
tab7 = VBox(children = ats.children)

tab = widgets.Tab(children=[tab1, tab2, tab3, tab4, tab5, tab6, tab7])

tab.set_title(0, 'Aggregate')
tab.set_title(1, 'Gross Range')
tab.set_title(2, 'Climatology')
tab.set_title(3, 'Flat Line')
tab.set_title(4, 'Spike')
tab.set_title(5, 'Rate of Change')
tab.set_title(6, 'Attenuated Signal')

tab

# Save Config

In [ ]:
with open(config_file, 'w') as f:
    json.dump(qc_config, f)
    print("New Config saved!")

# Save Results 

In [ ]:
#save file to csv
resultsFilePath = basedir.joinpath('Results/'+dataset_id+'_'+variable_name+'.csv')
data_full.to_csv(resultsFilePath)
print('Results file successfully saved. Available at '+ str(resultsFilePath)+"'.")